In [2]:
import string
import sqlite3
import random
import tensorflow as tf
# import keras
import numpy as np
import re
from unicodedata import normalize

(1) Definir el origen de los datos de entrenamiento

In [4]:
Base_de_datos = sqlite3.connect("Base de datos de Beige mejorada.db", check_same_thread=False)
cursor = Base_de_datos.cursor()
bbdd = cursor.execute("SELECT entrada,salida FROM oración")
# with open("fragmento de el viejo y el mar.txt") as f:
#     bbdd = f.read()

#Ahora crearemos un diccionario que definirá todas las conexiones entre las palabras en las respuestas vinculadas de cada una de las entradas
diccionario = {}
print("Creando diccionario...")
diccionario["__INICIO__"] = [] #creo la entrada inicial del diccionario. Esta es la lista de palabras que pueden iniciar la oración
try: #si lees desde una base de datos se ejecuta esta
    for respuesta in bbdd:
        palabras_en_oracion = respuesta[1] #si vas a activar el quitar signos de puntuacion(linea de abajo), sería bueno tambien ponerle .lower() a esto para que las palabras que iban despues de un punto ya no estén mayusculas
        # palabras_en_oracion = re.sub(r'[^\w\s]', '', palabras_en_oracion) #esto de aquí quita los signos de puntuación... no me parece necesario
        palabras_en_oracion = palabras_en_oracion.split()
        for indice, palabra in enumerate(palabras_en_oracion):
            if indice == 0:
                diccionario["__INICIO__"].append(palabra)

            if indice < len(palabras_en_oracion) - 1 :
                siguiente_palabra = palabras_en_oracion[indice+1]
            else:
                siguiente_palabra = "__FIN__"
            
            if palabra not in diccionario:
                diccionario[palabra] = []
                diccionario[palabra].append(siguiente_palabra)
            elif palabra in diccionario:
                diccionario[palabra].append(siguiente_palabra)
except: #si lees desde un archivo de texto fallará la lectura dede base de datos, ejecutemos esta
    palabras_en_oracion = bbdd
    # palabras_en_oracion = re.sub(r'[^\w\s]', '', palabras_en_oracion) #esto de aquí quita los signos de puntuación... no me parece necesario
    palabras_en_oracion = palabras_en_oracion.split()
    for indice, palabra in enumerate(palabras_en_oracion):
        if indice == 0:
            diccionario["__INICIO__"].append(palabra)

        if indice < len(palabras_en_oracion) - 1:
            siguiente_palabra = palabras_en_oracion[indice+1]
        else:
            siguiente_palabra = "__FIN__"

        if palabra not in diccionario:
            diccionario[palabra] = []
            diccionario[palabra].append(siguiente_palabra)
        elif palabra in diccionario:
            diccionario[palabra].append(siguiente_palabra)
print("Diccionario creado")
# print(diccionario)

Creando diccionario...
Diccionario creado


(2) Preparación de datos en forma numerica

(3) Preparación y entrenamiento del modelo

In [5]:
#obtención y preparación de datos
letras_a_numeros = {"a": 1, "b": 2, "c": 3, "d": 4, "e": 5, "f": 6, "g": 7, "h": 8, "i": 9, "j": 10,
               "k": 11, "l": 12, "m": 13, "n": 14, "ñ": 15, "o": 16, "p": 17, "q": 18, "r": 19, "s": 20, "t": 21, "u": 22, "v": 23, "w": 24, "x": 25, "y": 26, "z": 27}
bbdd = cursor.execute("SELECT entrada,salida FROM oración")
print("empezando")
lista_entrada = []
lista_salida = []

mas_larga = 0 
for fila in bbdd:
    entrada = fila[0].lower()
    entrada = re.sub(r'[^\w\s]', '', entrada)  # quito signos de puntuacion
    #quitamos tildes
    entrada = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
        normalize("NFD", entrada), 0, re.I
    ) 
    entrada = normalize('NFC', entrada) #normalizo
    oracion = []
    for palabra in entrada.lower():
        if "ñ" in palabra:
            oracion.append(28.0)
        elif " " in palabra:
            oracion.append(0.0)
        else:
            try:
                oracion.append(float(string.ascii_lowercase.index(palabra)+1))
            except ValueError:
                oracion.append(float(palabra))
    if len(oracion) > mas_larga:
        mas_larga = len(oracion)
    lista_entrada.append(np.array(oracion, dtype = float))


    #respuestas de salida
    salida = fila[1].lower()
    # print(salida)
    salida = re.sub(r'[^\w\s]', '', salida)  # quito signos de puntuacion
    #quitamos tildes
    salida = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
        normalize("NFD", salida), 0, re.I
    ) 
    salida = normalize('NFC', salida) #normalizo
    oracion = []
    for palabra in salida.lower():
        if "ñ" in palabra:
            oracion.append(28.0)
        elif " " in palabra:
            oracion.append(0.0)
        else:
            try:
                oracion.append(float(string.ascii_lowercase.index(palabra)+1))
            except ValueError:
                oracion.append(float(palabra))
    if len(oracion) > mas_larga:
        mas_larga = len(oracion)
    lista_salida.append(np.array(oracion, dtype = float))

# print(lista_entrada)
# print(lista_salida)
# print(mas_larga)
print("terminado")

#ajustamos todos los arreglos que corresponden a oraciones para tener la misma longitud
for indice, arreglo in enumerate(lista_entrada):
    diferencia = mas_larga - len(arreglo)
    lista_entrada[indice] = np.append(arreglo, np.zeros(diferencia, dtype = float))

for indice, arreglo in enumerate(lista_salida):
    diferencia = mas_larga - len(arreglo)
    lista_salida[indice] = np.append(arreglo, np.zeros(diferencia, dtype = float))


empezando
terminado


Opciones de entrenamiento

1. Original hecho por mí, sin casi experiencia

In [9]:
#Creamos las capas de la red neuronal y establecemos su secuencia
capa_oculta1 = tf.keras.layers.Dense(units = 100, input_dim = mas_larga )
capa_oculta2 = tf.keras.layers.Dense(units = 150)
capa_oculta3 = tf.keras.layers.Dense(units = 200)
capa_oculta4 = tf.keras.layers.Dense(units = 70)
capa_oculta5 = tf.keras.layers.Dense(units = 50)
capa_oculta6 = tf.keras.layers.Dense(units = 10)
# capa_oculta6 = tf.keras.layers.Dense(units = 1000)
capa_salida = tf.keras.layers.Dense(units=mas_larga)
# capa_oculta2, capa_salida])
modelo = tf.keras.Sequential([capa_oculta1, capa_oculta2, capa_oculta3, capa_oculta4, capa_oculta5, capa_oculta6, capa_salida])

#compilo en modelo
modelo.compile(
    optimizer = tf.keras.optimizers.Adam(0.1),
    loss = "mean_squared_error",
)


# #entrenamiento del modelo
lista_entrada = np.array(lista_entrada) #keras solo trabaja con arreglos numpy
lista_salida = np.array(lista_salida) #keras solo trabaja con arreglos numpy

print("\nEntrenando...")
historia = modelo.fit(lista_entrada, lista_salida, epochs=1000, verbose=False)
print("Entrenado\nRespuestas:")

# predecir = np.array([100.0,200.0,300.0,400.0,500.0,600.0,700.0,800.0])
# diferencia = mas_larga - len(predecir)
# predecir = np.append(predecir, np.zeros(diferencia, dtype=float))

print( modelo.predict(lista_entrada) )


Entrenando...
Entrenado
Respuestas:
21/21 [==============================] - 0s 1ms/step
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


2. hecho por chat gpt en base a mi diseño anterior y luego modificado por mí y chatgpt otra vez

In [17]:
import tensorflow as tf
import numpy as np

# Configuración del modelo
capas_ocultas = [100, 150, 200, 70, 50, 10]

# Crear capas ocultas con dropout
capas = [tf.keras.layers.Dense(units=capas_ocultas[0], input_dim=mas_larga, activation='relu')]
for i in range(1, len(capas_ocultas)):
    capa = tf.keras.layers.Dense(units=capas_ocultas[i], activation='relu')
    capas.append(capa)
    capas.append(tf.keras.layers.Dropout(0.2))

# Crear modelo secuencial y agregar capas
from tensorflow.keras import regularizers

modelo = tf.keras.Sequential([
    tf.keras.layers.Dense(units=capas_ocultas[0], input_dim=mas_larga, activation='relu', 
                          kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2)
])

for i in range(1, len(capas_ocultas)):
    capa = tf.keras.layers.Dense(units=capas_ocultas[i], activation='relu',
                                 kernel_regularizer=regularizers.l2(0.01))
    modelo.add(capa)
    modelo.add(tf.keras.layers.Dropout(0.2))

modelo.add(tf.keras.layers.Dense(units=mas_larga))

modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
               loss="categorical_crossentropy",
               metrics=['accuracy'])

historia = modelo.fit(lista_entrada, lista_salida, epochs=100, verbose=2, validation_split=0.2)

print(modelo.predict(lista_entrada))


# Predicción
print(modelo.predict(lista_entrada))


Epoch 1/100
17/17 - 2s - loss: 4231.8960 - accuracy: 0.0133 - val_loss: 3750.0708 - val_accuracy: 0.0152 - 2s/epoch - 128ms/step
Epoch 2/100
17/17 - 0s - loss: 5090.6318 - accuracy: 0.0303 - val_loss: 1801.0952 - val_accuracy: 0.0152 - 107ms/epoch - 6ms/step
Epoch 3/100
17/17 - 0s - loss: 4974.6284 - accuracy: 0.0379 - val_loss: 3649.9827 - val_accuracy: 0.0152 - 176ms/epoch - 10ms/step
Epoch 4/100
17/17 - 0s - loss: 5033.0474 - accuracy: 0.0303 - val_loss: 3582.1516 - val_accuracy: 0.0152 - 210ms/epoch - 12ms/step
Epoch 5/100
17/17 - 0s - loss: 4970.3359 - accuracy: 0.0265 - val_loss: 3482.5481 - val_accuracy: 0.0152 - 196ms/epoch - 12ms/step
Epoch 6/100
17/17 - 0s - loss: 4796.4238 - accuracy: 0.0246 - val_loss: 3433.5032 - val_accuracy: 0.0152 - 197ms/epoch - 12ms/step
Epoch 7/100
17/17 - 0s - loss: 4555.2832 - accuracy: 0.0265 - val_loss: 2137.0920 - val_accuracy: 0.0152 - 190ms/epoch - 11ms/step
Epoch 8/100
17/17 - 0s - loss: 3960.6638 - accuracy: 0.0322 - val_loss: 2404.7637 - va

(4) Ejecuciones de prueba

In [18]:
entrada = ""
while entrada != "fin":
    print("\n")
    entrada = input("Hablame: ")
    palabra_obtenida = "__INICIO__"
    limite = 280
    oracion = ""
    while len(oracion) < limite:
        numero = random.randint(0, len(diccionario[palabra_obtenida])-1)
        palabra_obtenida = diccionario[palabra_obtenida][numero]
        if palabra_obtenida == "__FIN__":  
            break
        oracion += " " + palabra_obtenida
    
    print("dijiste: {0}".format(entrada))
    print("mi respuesta es: {0}".format(oracion))




dijiste: hola, saludos, buenos días
mi respuesta es:  Se domestíca


dijiste: hola
mi respuesta es:  Sí! Muchas gracias, sí


dijiste: hola
mi respuesta es:  Dale, enséñame


dijiste: hola
mi respuesta es:  Me tienes, es un poco la creo, algo de notificaciones


dijiste: hola
mi respuesta es:  Pues aquí hay algo enredada ... Lo sé. Yo voy a mí eso es malo, diosito te duele reírte?


dijiste: hola
mi respuesta es:  Hombre con una cosa ni una palabra de niveles y las personas están durmiendo?


dijiste: hola
mi respuesta es:  Las nuevas metas y profundo


dijiste: adios
mi respuesta es:  X2


dijiste: adios
mi respuesta es:  Qué nos lo mejor


dijiste: adios
mi respuesta es:  Ah sí?


dijiste: adios
mi respuesta es:  Me tienes, es el amor de mí eso a escucharlo, detenidamente, notarás que necesitaba para qué es la escribió una respuesta


dijiste: adios
mi respuesta es:  Lo sé... Era por la tengas me tienes razón .-.


dijiste: adios
mi respuesta es:  I feel ashamed


dijiste: adios
mi